In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

import numpy as np
import matplotlib.pyplot as plt
import os

print(tf.__version__)
print(keras.__version__)

2.1.0
2.2.4-tf


## 1. Set hyper parameters

In [2]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

tf.random.set_seed(777)

#### + Creating a checkpoint directory

In [3]:
cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'mnist_cnn_func'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_prefix = os.path.join(ckpt_dir_name, model_dir_name)

## 2. Make a data pipelining

In [4]:
mnist = keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

train_images = X_train.astype(np.float32) / 255.
test_images = X_test.astype(np.float32) / 255.

train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)

train_labels = to_categorical(y_train, num_classes=10)
test_labels = to_categorical(y_test, num_classes=10)

train_dataset = tf.data.Dataset.from_tensor_slices(
                    (train_images, train_labels)).\
                    shuffle(buffer_size=100000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices(
                    (test_images, test_labels)).\
                    batch(batch_size)

## 3. Build a neural network model

In [5]:
def create_model():
    inputs = keras.Input(shape=(28, 28, 1))
    conv1 = keras.layers.Conv2D(filters=32, kernel_size=3,
                                strides=1, padding='same',
                                activation=tf.nn.relu)(inputs)
    pool1 = keras.layers.MaxPool2D(pool_size=2, padding='same')(conv1)
    conv2 = keras.layers.Conv2D(filters=64, kernel_size=3,
                                strides=1, padding='same',
                                activation=tf.nn.relu)(pool1)
    pool2 = keras.layers.MaxPool2D(pool_size=2, padding='same')(conv2)
    conv3 = keras.layers.Conv2D(filters=128, kernel_size=3,
                                strides=1, padding='same',
                                activation=tf.nn.relu)(pool2)
    pool3 = keras.layers.MaxPool2D(pool_size=2, padding='same')(conv3)
    
    pool3_flat = keras.layers.Flatten()(pool3)
    dense4 = keras.layers.Dense(units=256,
                                activation=tf.nn.relu)(pool3_flat)
    drop4 = keras.layers.Dropout(0.4)(dense4)
    logits = keras.layers.Dense(units=10,
                                activation=tf.nn.relu)(drop4)
    return keras.Model(inputs=inputs, outputs=logits)

model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 128)         0     

## 4. Define a loss function

In [6]:
@tf.function
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(
                tf.keras.losses.categorical_crossentropy(
                    y_true=labels, y_pred=logits, from_logits=True))
    return loss

## 5. Calculate a gradient

In [7]:
@tf.function
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

## 6. Select an optimizer

In [8]:
optimizer = tf.keras.optimizers.Adam(learning_rate)

## 7. Define a metric for model's performance

In [9]:
@tf.function
def evaluate(model, images, labels):
    logits = model(images, training=False)
    correct_prediction = tf.equal( tf.argmax(logits, axis=1), 
                                   tf.argmax(labels, axis=1) )
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,
                                      dtype=tf.float32))
    return accuracy

## 8. Make a checkpoint for saving

In [10]:
checkpoint = tf.train.Checkpoint(cnn=model)

## 9. Train and Validate a neural network model

In [11]:
@tf.function
def train(model, images, labels):
    grads = grad(model, images, labels)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))


print('Learning started. It takes some time.')
for epoch in range(training_epochs):
    avg_loss = 0. ; avg_train_acc = 0. ; avg_test_acc = 0.
    train_step = 0 ; test_step = 0
    
    for images, labels in train_dataset:
        train(model, images, labels)
        
        loss = loss_fn(model, images, labels)
        avg_loss = avg_loss + loss
        
        acc = evaluate(model, images, labels)
        avg_train_acc = avg_train_acc + acc
        
        train_step += 1
        pass
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in test_dataset:
        acc = evaluate(model, images, labels)
        avg_test_acc = avg_test_acc + acc
        test_step += 1
        pass
    avg_test_acc = avg_test_acc / test_step
    
    print(f'Epoch: {epoch+1:3}  |  loss = {avg_loss:10.8f}  |  ' +
          f'train accuracy = {avg_train_acc:6.4f}  |  ' +
          f'test accuracy = {avg_test_acc:6.4f}')
    checkpoint.save(file_prefix=checkpoint_prefix)
    pass

print('Learning Finished !')

Learning started. It takes some time.
Epoch:   1  |  loss = 0.17413007  |  train accuracy = 0.9554  |  test accuracy = 0.9870
Epoch:   2  |  loss = 0.03872857  |  train accuracy = 0.9903  |  test accuracy = 0.9890
Epoch:   3  |  loss = 0.02697025  |  train accuracy = 0.9934  |  test accuracy = 0.9927
Epoch:   4  |  loss = 0.01885665  |  train accuracy = 0.9953  |  test accuracy = 0.9923
Epoch:   5  |  loss = 0.01468997  |  train accuracy = 0.9964  |  test accuracy = 0.9921
Epoch:   6  |  loss = 0.01185038  |  train accuracy = 0.9975  |  test accuracy = 0.9925
Epoch:   7  |  loss = 0.00863728  |  train accuracy = 0.9981  |  test accuracy = 0.9940
Epoch:   8  |  loss = 0.00786343  |  train accuracy = 0.9982  |  test accuracy = 0.9921
Epoch:   9  |  loss = 0.00617698  |  train accuracy = 0.9988  |  test accuracy = 0.9903
Epoch:  10  |  loss = 0.00511709  |  train accuracy = 0.9988  |  test accuracy = 0.9913
Epoch:  11  |  loss = 0.00451507  |  train accuracy = 0.9990  |  test accuracy = 0